In [2]:
from symbols import resnet50, resnet50_hybrid, VGG
from chips import focus_branch as fb
from utils.utils import *
from utils.coco_af import *
from det import ssd_module as det_module
from det.anchor_params import *

ModuleNotFoundError: No module named 'symbols'

In [3]:
import matplotlib.pyplot as plt
import mxnet as mx, cv2 as cv
from mxnet.gluon import nn
from mxnet import nd, sym, autograd, image
from mxnet.gluon import loss as gloss
import argparse, time, os
from gluoncv import model_zoo, data, utils

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [16]:
def focustest(fname, isize, net):
    frame = cv.imread(fname)
    img = nd.array(frame)
    feature = image.imresize(img, isize, isize).astype('float32')
    X = feature.transpose((2, 0, 1)).expand_dims(axis=0)
    res = net(X.as_in_context(mx.gpu()))
    plt.subplot(121)
    plt.imshow(res.asnumpy()[0, 0, :, :]);plt.title("Focus Pixels")
    plt.subplot(122)
    plt.imshow(cv.resize(res.asnumpy()[0, 0, :, :], (isize, isize))
               * nd.sum(X[0, :, :, :], axis=0).asnumpy() / 3, cmap='gray');
    plt.title("Heatmap")
    plt.show()

In [17]:
prenet = model_zoo.resnet50_v1b(pretrained=False, ctx=mx.gpu())
basenet = nn.HybridSequential()
for layers in prenet._children:
    basenet.add(prenet._children[layers])
basenet = basenet[0:-3]

In [39]:
net = nn.HybridSequential()
net.add(
    basenet,
    fb.HybridFocusBranch()
)
net[1].initialize(ctx=mx.gpu())
net[1].collect_params().setattr('lr_mult', 10)
net.hybridize()


In [1]:
model_path = "/home/cunyuan/code/pycharm/params/autofocus/"
net.load_parameters(model_path + "Focuser-is256e50bs08-pResNet50-dUSC1479raw-lr0.01x10")
fname = "/home/cunyuan/code/pycharm/data/uav/usc/2666/img/2456.jpg"
isize = 1024

"""
# for focus test


def foctest(vid = 2666, im = "0001",dp="/home/cunyuan/code/pycharm/data/uav/usc/", 
    isize=1024, gs=1, thr=0.6, IF_ABS=True):    
    fname = dp + "%s/"%vid +"img/%s.jpg"%im 
    frame = cv.imread(fname)
    img = nd.array(frame)
    img = image.imresize(img, isize, isize)
    feature = img.astype('float32')
    X = feature.transpose((2, 0, 1)).expand_dims(axis=0)
    fmap = net(X.as_in_context(mx.gpu()));
    conn = calcConnect(fmap[0,0,:,:].asnumpy(), gau_sigma = gs, thres_ratio = thr, IF_ABS=IF_ABS)
    
    plt.figure(1)
    plt.subplot(221);plt.imshow(fmap[0,0,:,:].asnumpy());plt.xlabel("(a)");
    plt.subplot(222);plt.imshow(conn[1]);plt.xlabel("(b)");
    plt.subplot(223);plt.xlabel("(c)");
    plt.imshow(cv.resize(fmap[0, 0, :, :].asnumpy(), (isize, isize))
        * nd.sum(X[0, :, :, :], axis=0).asnumpy() / 3, cmap="gray")
    plt.subplot(224);plt.xlabel("(d)");
    plt.imshow(cv.resize(conn[1].astype(np.float32), (isize, isize)) 
        * nd.sum(X[0, :, :, :], axis=0).asnumpy() / 3, cmap="gray")
    plt.figure(2);plt.imshow(nd.sum(X[0, :, :, :], axis=0).asnumpy() / 3, cmap="gray")
    
"""

NameError: name 'net' is not defined

In [41]:
class detnet(nn.HybridBlock):
    def __init__(self, base, bfoc, bdet, lr_mult = {"focus": 10,"detection": 20}, **kwargs):
        super().__init__()
        self.basenet = base
        self.focus_branch = bfoc
        self.detect_branch = bdet
    def hybrid_forward(self, F, x, rds, *args, **kwargs):
        focus_area = self.basenet(x)
        for k in rds:
            det_this_scl = self.detect_branch(focus_area)
            focus_area = self.focus_branch(self.basenet(x))
        return (focus_area, det_this_scl)

In [42]:
dnet = detnet(net[0], net[1], det_module.LightSSD(1, num_ach = num_anchors))

In [43]:
net[0][0].weight.data()


[[[[-3.61405239e-02 -7.70909563e-02 -3.60333472e-02 ...  1.16046652e-01
     1.09734587e-01  6.82049766e-02]
   [-9.06419680e-02 -1.34431809e-01 -8.96056816e-02 ...  1.62349567e-01
     2.15554863e-01  1.67108506e-01]
   [-8.72862041e-02 -1.65748253e-01 -1.63021043e-01 ...  1.24957405e-01
     2.46810555e-01  1.97816938e-01]
   ...
   [-4.81220484e-02 -1.64303705e-01 -1.98874027e-01 ...  1.67468488e-02
     2.23766834e-01  2.71252185e-01]
   [-1.29605206e-02 -1.37048289e-01 -1.82760909e-01 ... -3.37419286e-02
     1.24600373e-01  2.22173080e-01]
   [-2.33686548e-02 -1.24447115e-01 -1.82959720e-01 ... -8.25145319e-02
     6.01674579e-02  1.53561145e-01]]

  [[ 1.01904422e-02  8.66017211e-03 -1.88103989e-02 ... -4.14647870e-02
    -3.54149379e-02 -1.06931292e-02]
   [ 2.06666943e-02  3.60231437e-02  1.62141342e-02 ... -4.65017632e-02
    -4.74813394e-02 -3.81768905e-02]
   [ 3.21533345e-02  5.95741160e-02  3.19622159e-02 ... -5.32255322e-02
    -5.56481183e-02 -5.20227849e-02]
   ...
  

In [44]:
del(net)
dnet._children["basenet"][0].weight.data()


[[[[-3.61405239e-02 -7.70909563e-02 -3.60333472e-02 ...  1.16046652e-01
     1.09734587e-01  6.82049766e-02]
   [-9.06419680e-02 -1.34431809e-01 -8.96056816e-02 ...  1.62349567e-01
     2.15554863e-01  1.67108506e-01]
   [-8.72862041e-02 -1.65748253e-01 -1.63021043e-01 ...  1.24957405e-01
     2.46810555e-01  1.97816938e-01]
   ...
   [-4.81220484e-02 -1.64303705e-01 -1.98874027e-01 ...  1.67468488e-02
     2.23766834e-01  2.71252185e-01]
   [-1.29605206e-02 -1.37048289e-01 -1.82760909e-01 ... -3.37419286e-02
     1.24600373e-01  2.22173080e-01]
   [-2.33686548e-02 -1.24447115e-01 -1.82959720e-01 ... -8.25145319e-02
     6.01674579e-02  1.53561145e-01]]

  [[ 1.01904422e-02  8.66017211e-03 -1.88103989e-02 ... -4.14647870e-02
    -3.54149379e-02 -1.06931292e-02]
   [ 2.06666943e-02  3.60231437e-02  1.62141342e-02 ... -4.65017632e-02
    -4.74813394e-02 -3.81768905e-02]
   [ 3.21533345e-02  5.95741160e-02  3.19622159e-02 ... -5.32255322e-02
    -5.56481183e-02 -5.20227849e-02]
   ...
  